In [6]:
textiles = textile.Textiles('dataframe.csv')

## Задачи исследования
1. Отличаются ли пелены от бинтов по параметрам?
2. Отличаются ли узкие, средние и широкие бинты друг от друга по параметрам?
3. Каков процент рогожки?
4. Какие группы тканей по качеству получаются?
5. Различаются ли нити основы и нити утка?
6. Какое количество (процент) тканей высокого качества имеют сбалансированное переплетение?
7. Какое количество (процент) тканей среднего качества имеет сбалансированное переплетение?
8. Какое количество (процент) тканей ниского качества имеет сбалансированное переплетение?
9. Каково соотношение сбалансированных тканей и тканей с преобладанием нитей основы?
10. Высчитать "коэффициент заполняемости" тканей
11. Разделить ткани по сбалансированности (сбалансированные - ±3 нити; ткани с преобладанием нитей основы); рассчитать процент каждой группы

### Описание датафрейма
1. 'internal_storage' - номер по каталогу
2. 'grave' - номер могилы
3. 'warp_size' - размер изделия по основе
4. 'weft_size' - размер изделия по утку
5. 'type' - тип изделия
6. 'condition' - сохранность изделия
7. 'weaving_technique'
8. 'warp_material' - материал нитей основы
9. 'weft_material' - материал нитей утка
10. 'warp_dyed' - окрашивание нитей основы
11. 'weft_dyed' - окрашивание нитей утка
12. 'warp_twist' -  крутка нитей основы
13. 'weft_twist' - крутка нитей утка
14. 'warp_angle'
15. 'weft_angle'
16. 'warp_a' - толщина нитей основы (нижний предел)
17. 'warp_b' - толщина нитей основы (верхний предел)
18. 'weft_a' - толщина нитей утка (нижний предел)
19. 'weft_b' - толщина нитей утка (верхний предел)
20. 'warp_dens' - плотность по основе
21. 'weft_dens' - плотность по утку  
* * *
Числовые параметры - warp_size, wef_size, warp_a, warp_b, weft_a, weft_b, warp_mean, weft_mean, warp_dens, weft_dens

## Формирование основного датафрейма

In [7]:
warp_mean_quantile_1, warp_mean_quantile_3 = dataframe.warp_mean.quantile([0.25, 0.75])
weft_mean_quantile_1, weft_mean_quantile_3 = dataframe.weft_mean.quantile([0.25, 0.75])
warp_dens_quantile_1, warp_dens_quantile_3 = dataframe.warp_dens.quantile([0.25, 0.75])
weft_dens_quantile_1, weft_dens_quantile_3 = dataframe.weft_dens.quantile([0.25, 0.75])

# Исходный объем датафрейма
dataframe_length = len(dataframe)
print(dataframe_length)

# Очистка от "выбросов"; Формируется новый датафрейм - очищенный от выбросов - dataframe_cleared;
dataframe_cleared = dataframe[dataframe.warp_mean < (warp_mean_quantile_3 + 1.5 * (warp_mean_quantile_3 - warp_mean_quantile_1))]
dataframe_cleared = dataframe_cleared[dataframe_cleared.warp_mean > (warp_mean_quantile_1 - 1.5 * (warp_mean_quantile_3 - warp_mean_quantile_1))]
dataframe_cleared = dataframe_cleared[dataframe_cleared.weft_mean < (weft_mean_quantile_3 + 1.5 * (weft_mean_quantile_3 - weft_mean_quantile_1))]
dataframe_cleared = dataframe_cleared[dataframe_cleared.weft_mean > (weft_mean_quantile_1 - 1.5 * (weft_mean_quantile_3 - weft_mean_quantile_1))]

NameError: name 'dataframe' is not defined

## Формирование датафреймов по группам тканей

In [ ]:
# Выборка данных из подготовленного датафрейма;
# Предполагается, что для сбалансированных тканей разница между плотностью
# по основе и плотностью по утку не должна превышать 3 единиц;
balanced = dataframe[abs(dataframe['warp_dens'] - dataframe['weft_dens']) <= 3]
unbalanced = dataframe[abs(dataframe['warp_dens'] - dataframe['weft_dens']) > 3]

# Ткани с переплетением типа "рогожка";
baskets = dataframe[dataframe['weaving_technique'] == 'basket']

# выборка узких бинтов;
narrow_bands = dataframe[dataframe['type'] == 'n_band']

# выборка средних бинтов;
middle_bands = dataframe[dataframe['type'] == 'm_band']
shapiro_middle_bands_warp_mean = stats.shapiro(middle_bands.warp_mean)
print(shapiro_middle_bands_warp_mean)

# выборка широких бинтов;
wide_bands = dataframe[dataframe['type'] == 'w_band']

# все бинты (всех типов)
all_bands = pandas.concat([narrow_bands, middle_bands, wide_bands])

### Описательные статистики для полного датафрейма

## Графики распределения числовых параметров

### 1 - Толщина нитей основы (усредненное значение)

In [ ]:
stats.probplot(dataframe_cleared['warp_mean'], dist="norm", plot=pylab)
pylab.show()
seaborn.distplot(dataframe_cleared['warp_mean'])

In [ ]:
seaborn.boxplot(dataframe_cleared['warp_mean'])

In [ ]:
shapiro_warp_mean = stats.shapiro(dataframe_cleared.warp_mean.values)
shapiro_warp_mean_p_coefficient = shapiro_warp_mean[1]
print('{:.20f}'.format(shapiro_warp_mean_p_coefficient))
if shapiro_warp_mean_p_coefficient > 0.05:
    print('True')
else:
    print('False')

### 2 - Толщина нитей утка (усредненное значение)

In [ ]:
stats.probplot(dataframe.weft_mean, dist="norm", plot=pylab)
pylab.show()
seaborn.distplot(dataframe['weft_mean'])

In [ ]:
seaborn.boxplot(dataframe_cleared['weft_mean'])

In [ ]:
shapiro_weft_mean = stats.shapiro(dataframe_cleared['weft_mean'].values)
shapiro_weft_mean_p_coefficient = shapiro_weft_mean[1]
print('{:.20f}'.format(shapiro_weft_mean_p_coefficient))
if shapiro_weft_mean_p_coefficient > 0.05:
    print('True')
else:
    print('False')

### 3 - Плотность по основе

In [ ]:
stats.probplot(dataframe_cleared.warp_dens, dist="norm", plot=pylab)
pylab.show()
seaborn.distplot(dataframe_cleared['warp_dens'])

In [ ]:
seaborn.boxplot(dataframe_cleared['warp_dens'])

In [10]:
shapiro_warp_dens = stats.shapiro(dataframe_cleared.warp_dens.values)
shapiro_warp_dens_p_coefficient = shapiro_warp_dens[1]
print('{:.20f}'.format(shapiro_warp_dens_p_coefficient))
if shapiro_warp_dens_p_coefficient > 0.05:
    print('True')
else:
    print('False')

NameError: name 'stats' is not defined

### 4 - Плотность по утку

In [ ]:
stats.probplot(dataframe_cleared.weft_dens, dist="norm", plot=pylab)
pylab.show()
seaborn.distplot(dataframe_cleared['weft_dens'])

In [ ]:
seaborn.boxplot(dataframe_cleared['weft_dens'])

In [ ]:
shapiro_weft_dens = stats.shapiro(dataframe_cleared.weft_dens.values)
shapiro_weft_dens_p_coefficient = shapiro_weft_dens[1]
print('{:.20f}'.format(shapiro_weft_dens_p_coefficient))
if shapiro_weft_dens_p_coefficient > 0.05:
    print('True')
else:
    print('False')

Распределение основных численных параметров значимо отличается от нормального =>  
        применение методов параметрической статистики невозможно

### 1 - Отличаются ли пелены от бинтов по параметрам?
* * *
- Получить датафреймы содержащие только пелены и только бинты (все, независимо от типа)
- Рассчитать меры центральной тенденции для обоих наборов
* * *

### 2 - Отличаются ли узкие, средние и широкие бинты по параметрам?
* * *

### 3 - Какой процент рогожки?
* * *

### 4 - Какие группы тканей по качеству получаются?
* * *

### 5 - Различаются ли нити основы и нити утка?
* * *

### 6 - Какое количество (процент) тканей высокого качества имеют сбалансированное переплетение?
* * *

### 7 - Какое количество (процент) тканей среднего качества имеет сбалансированное переплетение?
* * *

### 8 - Какое количество (процент) тканей низкого качества имеет сбалансированное переплетение?
* * *

### 9 - Каково соотношение сбалансированных тканей и тканей с преобладанием нитей основы?
* * *

### 10 - Высчитать "коэффициент заполняемости" тканей
* * *

### 11 - Разделить ткани по сбалансированности (сбалансированные - ±3 нити; ткани с преобладанием нитей основы); рассчитать процент каждой группы
* * *